In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
       
    url= "https://www.glasdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword"+keyword+"&sc.keyword=data+Scientist&locT=&locId=&jobType="    #<-Modification made by Ken Jee

    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)
        time.sleep(.1)
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("react-job-listing")  #looks like this works. Buy since I don't understand what I'm doing....
        for job_button in job_buttons:  
                
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))

            if len(jobs) >= num_jobs:
                break
  
            job_button.click()  #You might 
     
            time.sleep(1)
    
            collected_successfully = False
 
            try:
                driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.    <- Again Ken Jee magic
            except NoSuchElementException:
                pass
            
            while not collected_successfully:

                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "css-1j389vi e1tk4kwz2")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                    
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-1hbqxax e1wijj240"]').text#    <- Again Ken Jee magic
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@class="css-1m5m32b e1tk4kwz4"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
#             #clicking on this:
#             try:
#                 driver.find_element_by_xpath('.//div[@data-item="tab" and @data-tab-type="overview"]').click()


            try:
                size = driver.find_element_by_xpath('.//span[text()="Size"]//following-sibling::*').text
            except NoSuchElementException:
                size = -1

            try:
                founded = driver.find_element_by_xpath('.//span[text()="Founded"]//following-sibling::*').text
            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = driver.find_element_by_xpath('.//span[text()="Type"]//following-sibling::*').text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = driver.find_element_by_xpath('.//span[text()="Industry"]//following-sibling::*').text
            except NoSuchElementException:
                industry = -1

            try:
                sector = driver.find_element_by_xpath('.//span[text()="Sector"]//following-sibling::*').text
            except NoSuchElementException:
                sector = -1

            try:
                revenue = driver.find_element_by_xpath('.//span[text()="Revenue"]//following-sibling::*').text
            except NoSuchElementException:
                revenue = -1


#             except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
#                 size = -1
#                 founded = -1
#                 type_of_ownership = -1
#                 industry = -1
#                 sector = -1
#                 revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//button[@data-test="pagination-next"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs) 

In [3]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 15, False, "./chromedriver", 4)
df

/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)
/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:28: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  job_buttons = driver.find_elements_by_class_name("react-job-listing")  #looks like this works. Buy since I don't understand what I'm doing....


Progress: 0/15


/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:43: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.    <- Again Ken Jee magic
/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:50: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:51: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:52: DeprecationW

Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


/var/folders/8n/wlhf0yh177gcscr6bvbzf8sm0000gn/T/ipykernel_17582/2152941891.py:149: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('.//button[@data-test="pagination-next"]').click()


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,Employer Provided Salary:$70 - $75 Per Hour,We are looking for a Data Scientist to analyze...,2.0,Altak Group\n2.0,Remote,1 to 50 Employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable
1,"Data Scientist, Associate",Employer Provided Salary:$100K,About this role\nWe’d love you to join our gro...,2.9,Media News Group\n2.9,Remote,10000+ Employees,1987,Company - Private,Publishing,Media & Communication,$500 million to $1 billion (USD)
2,Data Scientist,Employer Provided Salary:$55 - $65 Per Hour,Role: Data Scientist\nLocation: 100% Remote\nD...,4.0,ERPMark Inc\n4.0,Remote,1 to 50 Employees,-1,Company - Private,Business Consulting,Management & Consulting,Less than $1 million (USD)
3,Data Scientist,Employer Provided Salary:$100K - $130K,Tasks (illustrative) – Forecasts and models to...,-1,"3G Federal Solutions, LLC",Remote,Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
4,Sr IT Data Scientist,Employer Provided Salary:$65 - $70 Per Hour,Job Description:\nThe IT Data Scientist positi...,4.2,Brilliant Infotech\n4.2,Remote,51 to 200 Employees,2007,Company - Private,Information Technology Support Services,Information Technology,$5 to $10 million (USD)
5,Data Scientist,Employer Provided Salary:$40 Per Hour,We are looking for a Data Scientist to analyze...,1.8,Medical Products Laboratories\n1.8,Remote,Unknown,-1,Company - Private,Biotech & Pharmaceuticals,Pharmaceutical & Biotechnology,Unknown / Non-Applicable
6,Data Scientist,Employer Provided Salary:$42 - $53 Per Hour,Responsibilities\nBuilding tools to automate d...,4.4,intellipro\n4.4,Remote,201 to 500 Employees,-1,Company - Private,-1,-1,$10 to $25 million (USD)
7,Data Scientist,Employer Provided Salary:$65 - $75 Per Hour,Title: Data Scientist.\nLocation: Remote\nClie...,-1,Acord,Remote,-1,-1,-1,-1,-1,-1
8,Jr. Data Scientist,Employer Provided Salary:$75K - $90K, Apply Statistical and Machine Learning metho...,-1,Net2Aspire LLC,Remote,-1,-1,-1,-1,-1,-1
9,Data Scientist,Employer Provided Salary:$80K - $110K,We are actively looking for a Data Scientist t...,3.8,"Quality Counts, LLC\n3.8",Remote,51 to 200 Employees,2003,Company - Private,Research & Development,Management & Consulting,$5 to $10 million (USD)
